In [ ]:
#Importando a biblioteca Pandas
import pandas as pd

In [ ]:
#Carregando o arquivo(planilha):
try:
    df = pd.read_csv('credit_scoring_eng.csv')
except: 
    df= pd.read_csv('/datasets/credit_scoring_eng.csv')

In [ ]:
#Explorando o arquivo:
df.info()
df.head(10)

In [ ]:
#Inspecionando colunas, começando pela coluna 'children':
df['children'].value_counts()
#Ninguém pode ter -1 filhos, vou substituir este valor por 0, pois não sabemos ao certo a quantidade correta de filhos.

In [ ]:
#Substituindo valores -1 por 0 na coluna 'children'
df[df['children'] == -1] = 0  

In [ ]:
#Verificando coluna 'children' novamente, está corrigido.
df['children'].value_counts()

In [ ]:
#Inspecionando coluna 'days_employed':
df['days_employed'].value_counts()
#Existem valores negativos. Vou transformá-los em positivo, pois a pessoa que os inseriu pode ter inserido incorretamente.

In [ ]:
#Criando função para transformar números negativos em positivos:
def numero_positivo(valor):
    if valor <0:
        valor = valor * -1
    return valor
#Utilizando função para transformar números negativos em positivos:
df['days_employed'] = df['days_employed'].apply(numero_positivo)

In [ ]:
#Verificando números da coluna 'days_employed'.
df['days_employed'].value_counts()

In [ ]:
#Inspecionando coluna 'dob_years':
df['dob_years'].value_counts().sort_index(ascending=True)
#Ninguém que tem empréstimo no banco é menor do que 18 anos supostamente, e zero anos seria um bebê. Dado provavelmente incorreto.
#Vou substituir pela média de idade.

In [ ]:
#Descobrindo a média de idade:
dob_years_mean = df['dob_years'].mean()

In [ ]:
#Transformando valores iguais a zero em valores da média da coluna 'dob_years':
df[df['dob_years'] == 0] = dob_years_mean

In [ ]:
#Verificando se foi corrigido:
df['dob_years'].value_counts()
#Números ficaram em 'float', como se tivessem vírgula, vou alterar depois.

In [ ]:
#Verificando coluna 'education':
df['education'].value_counts()
#Coluna 'education' tem valores em letra minúscula e letra maiúscula, além de ter um número que não corresponde a nada.

In [ ]:
#Olhando mais de perto estas linhas
df[df['education'] == 43.2004181184669]
#Podemos excluir estas linhas

In [ ]:
#Excluindo do dataframe linhas da coluna 'education' com número 43.2004181184669.
df = df[df['education'] != 43.2004181184669]

In [ ]:
#Transformando todos valores da coluna 'education' em valores minúsculos
df['education'] = df['education'].str.lower()

In [ ]:
#Verificando valores da coluna 'education'
df['education'].value_counts()

In [ ]:
#Calculando valores duplicados
df.duplicated().sum()

Possíveis motivos dos valores duplicados:
1) O cliente pediu mais de um empréstimo
2) Perfis de clientes diferentes com os mesmos dados para todas colunas
3) Dados com letras minúsculas e maiúsculas

Não temos um ID de cliente que identifique exclusivamente os clientes. É possível que duas pessoas tenham os mesmos valores exatos para as demais colunas.
Por essa razão, não vou excluir duplicatas

Possíveis motivos dos valores ausentes:
1) A pessoa está atualmente desempregada
2) A pessoa mudou de emprego e o sistema do banco não está atualizado
3) Erro de quem inseriu as informações no banco de dados

In [ ]:
#Calculando valores ausentes
#Percebe-se valores ausentes nas duas colunas 'days_employed' e 'total_income'
df.isna().sum()

In [ ]:
#Verificando se as duas colunas estão relacionadas, de acordo com os valores ausentes.
filtered_df_days_employed = df[df['days_employed'].isna()]
filtered_df_total_income = df[df['total_income'].isna()]
filtered_df_days_employed_list = filtered_df_days_employed.index.to_list()
filtered_df_total_income_list = filtered_df_total_income.index.to_list()
#Verificando se são iguais
filtered_df_total_income_list == filtered_df_days_employed_list
#São iguais

In [ ]:
#Como são valores quantitativos, vou utilizar a média e a mediana para preencher os valores ausentes
#Vou utilizar a mediana para preencher os valores ausentes da coluna 'days_employed', pois são valores atípicos
df_days_employed_median = df['days_employed'].median()
df['days_employed'] = df['days_employed'].fillna(value = df_days_employed_median)

In [ ]:
#Vou utilizar a média para preencher os valores em total_income, pois não são valores atípicos
df_total_income_mean = df['total_income'].mean()
df['total_income'] = df['total_income'].fillna(value = df_total_income_mean)

In [ ]:
#Verificando se valores ausentes foram preenchidos
df.isna().sum()

In [ ]:
#Exibindo informações da tabela novamente
df.info()

In [ ]:
#Transformando valores 'float' em 'int':
df['days_employed'] = df['days_employed'].astype('int64')
df['dob_years'] = df['dob_years'].astype('int64')
df['education_id'] = df['education_id'].astype('int64')
df['family_status_id'] = df['family_status_id'].astype('int64')
df['debt'] = df['debt'].astype('int64')

In [ ]:
df.info()

In [ ]:
#Criando função para categorizar valores de acordo com o status familiar e a quantidade de filhos:
def potencial_credito(row):
    if row['children'] == 0 and row['family_status'] in ['married','civil partnership']:
        return 'Juntos sem filhos'
    if row['children'] == 0 and row['family_status'] in ['unmarried','divorced','widow / widower']:
        return 'Separado(a) sem filho'
    if row['children'] == 1 and row['family_status']in ['married','civil partnership']:
        return 'Juntos com 1 filho'
    if row['children'] == 1 and row['family_status'] in ['unmarried','divorced','widow / widower']:
        return 'Separado(a) com 1 filho'
    if row['children'] in [2,3,4,5] and row['family_status'] in ['married','civil partnership']:
        return 'Juntos com 2-5 filhos'
    if row['children'] in [2,3,4,5] and row['family_status'] in ['unmarried','divorced','widow / widower']:
        return 'Separado(a) com 2-5 filhos'
    if row['children'] == 20 and row['family_status'] in ['married','civil partnership']:
        return 'Juntos com 20 filhos'
    if row['children'] == 20 and row['family_status'] in ['unmarried','divorced','widow / widower']:
        return 'Separado(a) com 20 filhos'
#Aplicando função e criando nova coluna 'credit_potential':
df['credit_potential'] = df.apply(potencial_credito, axis = 1)
    

In [ ]:
#Criando tabela filtrada para respoder às perguntas:
df_debt_one = df[df['debt'] == 1]

In [ ]:
#Utilizando groupby para ajudar a analisar 
df_debt_one.groupby('credit_potential')['debt'].count().sort_values(ascending = True)

Existe alguma relação entre ter filhos e pagar um empréstimo em dia? Parece que quanto menos filhos, menor a chance de inadimplência.
Existe alguma relação entre o estado civil e o pagamento de um empréstimo no prazo estipulado? Não existe


In [ ]:
#Utilizando o groupby para ajudar a analisar
df_debt_one.groupby('credit_potential')['debt','total_income'].sum().sort_values(by='total_income',ascending=True)

Existe uma relação entre o nível de renda e o pagamento de um empréstimo no prazo? Parece que quanto mais 'income', maior a inadimplência.

In [ ]:
#Utilizando o groupby para ajudar a analisar
df.groupby('purpose')['debt'].count().sort_values(ascending=True)

Como as diferentes finalidades do empréstimo afetam o pagamento pontual do empréstimo? A maior parte dos empréstimos é utilizado para cerimônias de casamento e compra da própria casa

Conclusão geral: Casais com menos filhos tendem a pedir empréstimos para compras suas próprias casas e realizar seus casamentos, e não pagam os mesmos no prazo estipulado.